# Eksploracja Danych - Projekt
Tomasz Kiljańczyk (136257)

Wojciech Lulek (136280)

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import os

OUTPUT_FILE = "./data/stage_5_non_trending_ids.csv"
VIDEOS_PER_DAY = 200

## Date range preparation

In [ ]:
if os.path.exists(OUTPUT_FILE):
    non_trending_ids_df = pd.read_csv(OUTPUT_FILE, dtype={"video_id": str}, parse_dates=["publishedAt"])
    non_trending_ids_df.info()
else:
    non_trending_ids_df = pd.DataFrame(columns=["video_id", "publishedAt"])
    non_trending_ids_df["video_id"] = non_trending_ids_df["video_id"].astype(str)
    non_trending_ids_df["publishedAt"] = non_trending_ids_df["publishedAt"].astype(np.datetime64)

In [ ]:
us_trending_df = pd.read_csv("./data/stage_1_us_trending.csv", usecols=["video_id", "publishedAt"],
                             parse_dates=["publishedAt"])
us_trending_df.head()

In [ ]:
trending_ids = us_trending_df["video_id"].unique()

In [ ]:
end_date = us_trending_df["publishedAt"].dt.date.max()
end_date

In [ ]:
if non_trending_ids_df.shape[0] > 0:
    last_downloaded_date = non_trending_ids_df["publishedAt"].dt.date.max()
    last_downloaded_date += dt.timedelta(days=1)
else:
    last_downloaded_date = us_trending_df["publishedAt"].dt.date.min()

last_downloaded_date

## Download and save to CSV

In [ ]:
from youtube_api import download_non_trending_video_ids

if end_date == last_downloaded_date:
    print("All video ids downloaded")
else:
    while last_downloaded_date < end_date:
        video_ids = download_non_trending_video_ids(last_downloaded_date, VIDEOS_PER_DAY, trending_ids)
        video_ids_df_format = [{"video_id": video_id, "publishedAt": published_at} for video_id, published_at in
                               video_ids]

        non_trending_ids_df = non_trending_ids_df.append(video_ids_df_format, ignore_index=True)
        non_trending_ids_df.to_csv(OUTPUT_FILE, index=False)
        last_downloaded_date += dt.timedelta(days=1)